In [1]:
!pip install unicodecsv

  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp36-none-any.whl size=10768 sha256=863b7a188a8326051987813b2b2028f2b7ac726262885e6906906d7b10fc7338
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
Successfully built unicodecsv


In [2]:
!pip install distance

     |████████████████████████████████| 184kB 2.6MB/s 
  Created wheel for distance: filename=Distance-0.1.3-cp36-none-any.whl size=16261 sha256=a89ecbd7b33e76212a932929185d61db24db8f16a2a4c8ae162f37437e01838d
  Stored in directory: /root/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e
Successfully built distance


In [3]:
!pip install jellyfish

     |████████████████████████████████| 102kB 2.6MB/s 


In [4]:
from __future__ import division
from keras.models import load_model
import csv
import copy
import numpy as np
import distance
#from itertools import izip
from jellyfish._jellyfish import damerau_levenshtein_distance
import unicodecsv
from sklearn import metrics
from math import sqrt
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import Counter

Using TensorFlow backend.


In [0]:
eventlog = "/content/drive/My Drive/MSc Dissertation/Data/helpdesk.csv"

In [0]:
csvfile = open(eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161

In [0]:
lastcase = ''
line = ''
firstLine = True
lines = []
caseids = []
timeseqs = []
timeseqs2 = []
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [8]:
for row in spamreader: #the rows are "CaseID,ActivityID,CompleteTimestamp"
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        caseids.append(row[0])
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
        line = ''
        times = []
        times2 = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    #line+=chr(int(row[1]))
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    print(row[1],line,timediff,timediff2)
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    firstLine = False

Streaming output truncated to the last 5000 lines.
6 ¢©§§ 1 173931
1 ¢ 0 0
6 ¢§ 28 28
1 ¢ 0 0
8 ¢© 838 838
8 ¢©© 46 884
2 ¢©©£ 500028 500912
6 ¢©©£§ 4331204 4832116
1 ¢ 0 0
8 ¢© 21844 21844
9 ¢©ª 3909 25753
8 ¢©ª© 600034 625787
6 ¢©ª©§ 86501 712288
6 ¢©ª©§§ 2 712290
1 ¢ 0 0
8 ¢© 123 123
6 ¢©§ 62 185
1 ¢ 0 0
8 ¢© 17 17
6 ¢©§ 85104 85121
1 ¢ 0 0
1 ¢¢ 134 134
8 ¢¢© 156 290
6 ¢¢©§ 59368 59658
1 ¢ 0 0
1 ¢¢ 507756 507756
8 ¢¢© 7 507763
9 ¢¢©ª 536520 1044283
8 ¢¢©ª© 1471593 2515876
6 ¢¢©ª©§ 12 2515888
1 ¢ 0 0
8 ¢© 20429 20429
6 ¢©§ 31 20460
1 ¢ 0 0
8 ¢© 61 61
6 ¢©§ 202372 202433
8 ¢©§© 825 203258
9 ¢©§©ª 339610 542868
8 ¢©§©ª© 246280 789148
6 ¢©§©ª©§ 20 789168
3 ¤ 0 0
1 ¤¢ 72562 72562
8 ¤¢© 3770 76332
6 ¤¢©§ 397 76729
1 ¢ 0 0
8 ¢© 188197 188197
6 ¢©§ 415690 603887
1 ¢ 0 0
8 ¢© 98770 98770
6 ¢©§ 83510 182280
1 ¢ 0 0
8 ¢© 250245 250245
6 ¢©§ 10 250255
1 ¢ 0 0
8 ¢© 27 27
6 ¢©§ 429 456
1 ¢ 0 0
8 ¢© 14924 14924
9 ¢©ª 524 15448
8 ¢©ª© 1189293 1204741
6 ¢©ª©§ 10 1204751
8 ¢©ª©§© 184 1204935
6 ¢©ª©§©

In [0]:
# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1

In [10]:
divisor = np.mean([item for sublist in timeseqs for item in sublist])
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist])
print('divisor2: {}'.format(divisor2))

divisor: 210915.5199854121
divisor2: 409874.9012399708


In [0]:
elems_per_fold = int(round(numlines/3))
fold1 = lines[:elems_per_fold]
fold1_c = caseids[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]

In [0]:
fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_c = caseids[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]

In [0]:
lines = fold1 + fold2
caseids = fold1_c + fold2_c
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

In [0]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = map(lambda x: x+'!',lines)
lines = list(lines)
maxlen = max(map(lambda x: len(x),lines))

In [15]:
print(maxlen)

15


In [16]:
chars = map(lambda x : set(x),lines)
chars = list(set().union(*chars))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

total chars: 9, target chars: 10
{0: '¢', 1: '£', 2: '¤', 3: '¥', 4: '¦', 5: '§', 6: '¨', 7: '©', 8: 'ª'}


In [0]:
lastcase = ''
line = ''
firstLine = True
lines = []
caseids = []
timeseqs = []  # relative time since previous event
timeseqs2 = [] # relative time since case start
timeseqs3 = [] # absolute time of previous event
times = []
times2 = []
times3 = []
numlines = 0
casestarttime = None
lasteventtime = None

In [18]:
csvfile = open(eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers

['CaseID', 'ActivityID', 'CompleteTimestamp']

In [0]:
for row in spamreader:
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        caseids.append(row[0])
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:        
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
        line = ''
        times = []
        numlines+=1
    line+=chr(int(row[1])+ascii_offset)
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    #timediff = log(timediff+1)
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    lasteventtime = t
    firstLine = False

In [0]:
# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
numlines+=1

In [0]:
fold3 = lines[2*elems_per_fold:]
fold3_c = caseids[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
#fold3_t4 = timeseqs4[2*elems_per_fold:]

In [0]:
lines = fold3
caseids = fold3_c
lines_t = fold3_t
lines_t2 = fold3_t2
lines_t3 = fold3_t3
#lines_t4 = fold1_t4 + fold2_t4

In [0]:
# set parameters
predict_size = 1

In [0]:
# load model, set this to the model generated by train.py
model = load_model('/content/drive/My Drive/MSc Dissertation/Data/output_files/models/model_24-1.51.h5')

In [0]:
# define helper functions
def encode(sentence, times, times3, maxlen=maxlen):
    num_features = len(chars)+5
    X = np.zeros((1, maxlen, num_features), dtype=np.float32)
    leftpad = maxlen-len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t]-midnight
        multiset_abstraction = Counter(sentence[:t+1])
        for c in chars:
            if c==char:
                X[0, t+leftpad, char_indices[c]] = 1
        X[0, t+leftpad, len(chars)] = t+1
        X[0, t+leftpad, len(chars)+1] = times[t]/divisor
        X[0, t+leftpad, len(chars)+2] = times2[t]/divisor2
        X[0, t+leftpad, len(chars)+3] = timesincemidnight.seconds/86400
        X[0, t+leftpad, len(chars)+4] = times3[t].weekday()/7
    return X

In [0]:
def getSymbol(predictions):
    maxPrediction = 0
    symbol = ''
    i = 0;
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_char[i]
        i += 1
    return symbol

In [0]:
one_ahead_gt = []
one_ahead_pred = []

two_ahead_gt = []
two_ahead_pred = []

three_ahead_gt = []
three_ahead_pred = []

In [0]:
eventlog = 'helpdesk.csv'

In [29]:
# make predictions
with open('/content/drive/My Drive/MSc Dissertation/Data/output_files/results/next_activity_and_time_%s' % eventlog, 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["CaseID", "Prefix length", "Groud truth", "Predicted", "Levenshtein", "Damerau", "Jaccard", "Ground truth times", "Predicted times", "RMSE", "MAE"])
    for prefix_size in range(2,maxlen):
        print(prefix_size)
        for line, caseid, times, times3 in zip(lines, caseids, lines_t, lines_t3):
            times.append(0)
            cropped_line = ''.join(line[:prefix_size])
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            if '!' in cropped_line:
                continue # make no prediction for this case, since this case has ended already
            ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
            ground_truth_t = times[prefix_size:prefix_size+predict_size]
            predicted = ''
            predicted_t = []
            for i in range(predict_size):
                if len(ground_truth)<=i:
                    continue
                enc = encode(cropped_line, cropped_times, cropped_times3)
                y = model.predict(enc, verbose=0)
                y_char = y[0][0]
                y_t = y[1][0][0]
                prediction = getSymbol(y_char)              
                cropped_line += prediction
                if y_t<0:
                    y_t=0
                cropped_times.append(y_t)
                y_t = y_t * divisor
                cropped_times3.append(cropped_times3[-1] + timedelta(seconds=y_t))
                predicted_t.append(y_t)
                if i==0:
                    if len(ground_truth_t)>0:
                        one_ahead_pred.append(y_t)
                        one_ahead_gt.append(ground_truth_t[0])
                if i==1:
                    if len(ground_truth_t)>1:
                        two_ahead_pred.append(y_t)
                        two_ahead_gt.append(ground_truth_t[1])
                if i==2:
                    if len(ground_truth_t)>2:
                        three_ahead_pred.append(y_t)
                        three_ahead_gt.append(ground_truth_t[2])
                if prediction == '!': # end of case was just predicted, therefore, stop predicting further into the future
                    print('! predicted, end case')
                    break
                predicted += prediction
            output = []
            if len(ground_truth)>0:
                output.append(caseid)
                output.append(prefix_size)
                #output.append(str(ground_truth).encode("utf-8"))
                output.append(ord(ground_truth) - ascii_offset)
                #output.append(str(predicted).encode("utf-8"))
                if (len(predicted)>0): output.append(ord(predicted) - ascii_offset)
                output.append(1 - distance.nlevenshtein(predicted, ground_truth))
                dls = 1 - (damerau_levenshtein_distance(str(predicted), str(ground_truth)) / max(len(predicted),len(ground_truth)))
                if dls<0:
                    dls=0 # we encountered problems with Damerau-Levenshtein Similarity on some linux machines where the default character encoding of the operating system caused it to be negative, this should never be the case
                output.append(dls)
                output.append(1 - distance.jaccard(predicted, ground_truth))
                #print(ground_truth_t)
                output.append('; '.join(str(x) for x in ground_truth_t))
                output.append('; '.join(str(x) for x in predicted_t))
                if len(predicted_t)>len(ground_truth_t): # if predicted more events than length of case, only use needed number of events for time evaluation
                    predicted_t = predicted_t[:len(ground_truth_t)]
                if len(ground_truth_t)>len(predicted_t): # if predicted less events than length of case, put 0 as placeholder prediction
                    predicted_t.extend(range(len(ground_truth_t)-len(predicted_t)))
                if len(ground_truth_t)>0 and len(predicted_t)>0:
                    output.append('')
                    output.append(metrics.mean_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                    #output.append(metrics.median_absolute_error([ground_truth_t[0]], [predicted_t[0]]))
                else:
                    output.append('')
                    output.append('')
                    output.append('')
                spamwriter.writerow(output)

2
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
3
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! predicted, end case
! pred

In [0]:
# temp = 1
# x = chr(int(temp)+ascii_offset)
# print(x)
# x = str(x)
# print(x)
# y = ord(x) - ascii_offset
# print(y)

# Analyzing predictions

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('/content/drive/My Drive/MSc Dissertation/Data/output_files/results/next_activity_and_time_helpdesk.csv')

In [33]:
df.head()

,CaseID,Prefix length,Groud truth,Predicted,Levenshtein,Damerau,Jaccard,Ground truth times,Predicted times,RMSE,MAE
0,3087,2,6,6.0,1.0,1.0,1.0,256462.0,71556.069259,NaN,184905.930741
1,3088,2,8,8.0,1.0,1.0,1.0,14.0,10139.063901,NaN,10125.063901
2,3089,2,9,6.0,0.0,0.0,0.0,355698.0,77431.537794,NaN,278266.462206
3,3090,2,6,6.0,1.0,1.0,1.0,1800.0,39465.655249,NaN,37665.655249
4,3091,2,6,6.0,1.0,1.0,1.0,511775.0,30799.863237,NaN,480975.136763


In [0]:
# df = df[['CaseID','Prefix length','Groud truth', 'Predicted', 'Ground truth times', 'Predicted times']]
# df.head()

In [35]:
from sklearn.metrics import accuracy_score

accuracy_score(df['Groud truth'],df['Predicted'])

0.703963873557451

In [36]:
df['MAE'].apply(lambda x:x/86400).mean()

3.7071515481198936